### 1. Import

In [1]:
import os
import json

from langchain_openai import ChatOpenAI
from ragas.llms import LangchainLLMWrapper

from document_processor import TextProcessor, ImageProcessor, PageImageProcessor, TextAndInlineImageProcessor
from multimodal_rag import MultimodalRAG
from embedder import SFREmbedder, SigLIPEmbedder, VisRAGEmbedder, OpenAIEmbedder
from pdf_to_qa import generate_qa_for_pdf
from evaluation import evaluate_generation, compute_mrr_at_k
from datasets import load_dataset
from chartQAloader import generate_chartQA_pdf_and_json

#### 1.2 Load chartQA

In [2]:
dataset = load_dataset('lmms-lab/ChartQA', split='test')

In [4]:
generate_chartQA_pdf_and_json(dataset, pdf_output_path='ChartQA_Evaluation_Set.pdf', json_output_path='ChartQA_QA_Mapping.json')

KeyError: 'image_path'

### 2. Configuration

In [4]:
PDF_FILE = "knowledge/subset_riksbanken.pdf"

#text_processor = TextProcessor(SFREmbedder())
#image_processor = ImageProcessor(SigLIPEmbedder())
#page_image_processor = PageImageProcessor(VisRAGEmbedder())
#text_inline_processor = TextAndInlineImageProcessor(SFREmbedder())
text_inline_processor = TextAndInlineImageProcessor(OpenAIEmbedder())

rag = MultimodalRAG([text_inline_processor], PDF_FILE)
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini")) # For Ragas evaluation

### 3. Generate Dataset - for CoinQA

In [8]:
# Check if QA file already exists
qa_filepath = "QA_" + os.path.basename(PDF_FILE).replace('.pdf', '.json')

if os.path.exists(qa_filepath):
    qa_path = qa_filepath
    print(f"Using existing QA file: {qa_path}")
else:
    qa_path = generate_qa_for_pdf(PDF_FILE)
    print(f"Generated new QA file: {qa_path}")

with open(qa_path, 'r', encoding='utf-8') as f:
    qa_data = json.load(f)
    

Using existing QA file: QA_subset_riksbanken.json


### 4. Answering the QA

In [ ]:
with open(qa_path, 'r', encoding='utf-8') as f:
    qa_data = json.load(f)

# Generate dataset
rag_generated_answers = []

# Check if generated answers file already exists
rag_answers_path = "rag_generated_answers_" + os.path.basename(qa_path)

if os.path.exists(rag_answers_path):
    rag_generated_answers = json.load(open(rag_answers_path, 'r', encoding='utf-8'))
    print(f"Using existing RAG generated answers file: {rag_answers_path}")
else:
    for qa in qa_data:
        query = qa["question"]
        reference = qa["answer"]

        relevant_docs = rag.get_most_relevant_docs(query, top_k=10)
        response = rag.generate_answer(query, relevant_docs)
        rag_generated_answers.append(
            {
                "query":query,
                "retrieved_contexts":relevant_docs,
                "generated_answer":response,
                "true_answer":reference
            }
        )

    # Save the dataset to a JSON file
    output_dataset_file = "rag_generated_answers_" + os.path.basename(qa_path)

    with open(output_dataset_file, 'w', encoding='utf-8') as f:
        json.dump(rag_generated_answers, f, ensure_ascii=False, indent=4)
    print(f"Generated new RAG generated answers file: {output_dataset_file}")

Generated new RAG generated answers file: rag_generated_answers_QA_subset_riksbanken.json


### 5. Evaluate Retrieval

In [9]:
all_real_pages, all_retrieved_pages = [], []

for rag_answer in rag_generated_answers:
    real_page = next(
        item["page_number"] for item in qa_data if item["question"] == rag_answer["query"]
    )
    retrieved_pages = [doc["page_number"] for doc in rag_answer["retrieved_contexts"]]
    
    all_real_pages.append(real_page)
    all_retrieved_pages.append(retrieved_pages)

for k in [3, 5]:
    mrr = compute_mrr_at_k(all_retrieved_pages, all_real_pages, k)
    print(f"MRR@{k}: {mrr}")

MRR@3: 0.7
MRR@5: 0.75


### 6. Evaluate Generation

In [10]:
faithfulness_and_relevance = evaluate_generation(rag_generated_answers, evaluator_llm)
print(faithfulness_and_relevance)

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

{'faithful_rate': 1.0000, 'relevance_rate': 1.0000}
